In [29]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy 
import random
import requests
from config import weather_api_key
from datetime import datetime
import json
import time
import gmaps
from config import g_key

In [5]:
#Avoid error when loading heatmaps from gmaps
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [30]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [3]:
#Import data from Weather Database
city_data_df=pd.read_csv('data/WeatherPy_challenge.csv')
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Hilo,US,19.73,-155.09,78.80,61,1,4.70,clear sky,0.0,0.0
1,Te Anau,NZ,-45.42,167.72,46.06,79,3,3.09,clear sky,0.0,0.0
2,Agadez,NE,19.75,10.25,88.11,16,5,7.25,clear sky,0.0,0.0
3,Byron Bay,AU,-28.65,153.62,75.20,53,29,10.29,scattered clouds,0.0,0.0
4,Hobart,AU,-42.88,147.33,59.00,54,75,5.82,broken clouds,0.0,0.0


In [4]:
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [56]:
#Ask travel plan preferences
try:
    min_temp = float(input("What is the minimum temperature you would like for your trip? "))

except:
    print("Enter valid temperature")
try:
    max_temp = float(input("What is the maximum temperature you would like for your trip? "))
except:
    print("Enter valid temperature")
try:
    is_raining=str(input("Do you want it to be raining? (yes/no) "))
except:
    print("Please enter 'yes' or 'no' only")
try:
    is_snowing=str(input("Do you want it to be snowing? (yes/no) "))
    
except:
    print("Please enter 'yes' or 'no' only")
  

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) nnn


In [49]:
#Set dataframe based on criteria
temp_filter_df=city_data_df.loc[(city_data_df['Max Temp']<= max_temp)&\
                                     (city_data_df['Max Temp']>=min_temp)].copy().reset_index(drop=True)
# temp_filter_df

if is_raining=='yes' and is_snowing=='no':
    preferred_cities_df=temp_filter_df.loc[(temp_filter_df['Rain (inches)']>0) &\
                                          (temp_filter_df['Snow (inches)']==0)]
elif is_raining=='yes' and is_snowing=='yes':
    preferred_cities_df=temp_filter_df.loc[(temp_filter_df['Rain (inches)']>0) &\
                                          (temp_filter_df['Snow (inches)']>0)]
elif is_raining=='no' and is_snowing=='yes':
    preferred_cities_df=temp_filter_df.loc[(temp_filter_df['Rain (inches)']==0) &\
                                          (temp_filter_df['Snow (inches)']>0)] 
else:
    preferred_cities_df=temp_filter_df.loc[(temp_filter_df['Rain (inches)']==0) &\
                                          (temp_filter_df['Snow (inches)']==0)] 
preferred_cities_df.head(10)



,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Hilo,US,19.73,-155.09,78.80,61,1,4.70,clear sky,0.0,0.0
1,Agadez,NE,19.75,10.25,88.11,16,5,7.25,clear sky,0.0,0.0
2,Byron Bay,AU,-28.65,153.62,75.20,53,29,10.29,scattered clouds,0.0,0.0
3,Lakatoro,VU,-16.10,167.42,84.20,70,20,1.12,few clouds,0.0,0.0
4,Puerto Maldonado,PE,-12.60,-69.18,71.38,78,100,1.23,overcast clouds,0.0,0.0
5,Atuona,PF,-9.80,-139.03,80.92,79,12,18.19,few clouds,0.0,0.0
6,Puerto Ayora,EC,-0.74,-90.35,73.99,95,59,1.01,broken clouds,0.0,0.0
8,Rikitea,PF,-23.12,-134.97,76.10,81,53,9.10,broken clouds,0.0,0.0
9,Butaritari,KI,3.07,172.79,83.86,76,1,16.89,clear sky,0.0,0.0
10,Katsuura,JP,35.13,140.30,72.00,86,100,14.79,light rain,0.0,0.0


In [50]:
#Create dataframe to add the hotel name
hotel_df=preferred_cities_df[['City','Country','Max Temp','Current Description','Lat','Lng']]
hotel_df['Hotel Name']=''
hotel_df.head(10)

C:\Users\gamad\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,78.80,clear sky,19.73,-155.09,
1,Agadez,NE,88.11,clear sky,19.75,10.25,
2,Byron Bay,AU,75.20,scattered clouds,-28.65,153.62,
3,Lakatoro,VU,84.20,few clouds,-16.10,167.42,
4,Puerto Maldonado,PE,71.38,overcast clouds,-12.60,-69.18,
5,Atuona,PF,80.92,few clouds,-9.80,-139.03,
6,Puerto Ayora,EC,73.99,broken clouds,-0.74,-90.35,
8,Rikitea,PF,76.10,broken clouds,-23.12,-134.97,
9,Butaritari,KI,83.86,clear sky,3.07,172.79,
10,Katsuura,JP,72.00,light rain,35.13,140.30,


In [51]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found ... skipping")

hotel_df.head(10)

C:\Users\gamad\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,78.80,clear sky,19.73,-155.09,Hilo Hawaiian Hotel
1,Agadez,NE,88.11,clear sky,19.75,10.25,
2,Byron Bay,AU,75.20,scattered clouds,-28.65,153.62,Backpackers Inn on the Beach
3,Lakatoro,VU,84.20,few clouds,-16.10,167.42,Lakatoro Palm Lodge
4,Puerto Maldonado,PE,71.38,overcast clouds,-12.60,-69.18,Wasai Hotel
5,Atuona,PF,80.92,few clouds,-9.80,-139.03,Villa Enata
6,Puerto Ayora,EC,73.99,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
8,Rikitea,PF,76.10,broken clouds,-23.12,-134.97,Pension Maro'i
9,Butaritari,KI,83.86,clear sky,3.07,172.79,Isles Sunset Lodge
10,Katsuura,JP,72.00,light rain,35.13,140.30,Katsuura Hotel Mikazuki


In [52]:
# Create map with pop-up marker
#Define locations
locations=hotel_df[['Lat','Lng']]
#assign the figure variable
fig=gmaps.figure(center=(30.0,15.0),zoom_level=1.3)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
#add the layers
fig.add_layer(marker_layer)
#call the figure to plot
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
# Create the output file (CSV).
output_data_file='data/WeatherPy_vacation.csv'
# Export the City_data into a CSV
hotel_df.to_csv(output_data_file,index=False)